# Flask GUI for Graph Colouring Problem Solver


In [ ]:
%pip install flask networkx matplotlib



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from flask import Flask, render_template, request, jsonify
import networkx as nx
import matplotlib.pyplot as plt
import os
import io
import base64
from Backtracking import backtracking_colouring
from CulturalAlgorithm import CulturalAlgorithm

In [ ]:
from flask import Flask, render_template, request, jsonify
import networkx as nx
import matplotlib.pyplot as plt
import io
import base64
from Backtracking import find_optimal_backtracking
from BacktrackHeu import solve_graph_coloring
from CulturalAlgorithm import CulturalAlgorithm

app = Flask(__name__)

def calculate_conflicts(graph, individual):
    conflicts = 0
    n = len(graph)
    for i in range(n):
        if individual[i] == -1:
            continue
        for j in range(i + 1, n):
            if graph[i][j] == 1 and individual[i] == individual[j]:
                conflicts += 1
    return conflicts

def calculate_chromatic_number(solution):
    if solution is None:
        return 0
    assigned_colors = set(c for c in solution if c != -1)
    return len(assigned_colors)

def plot_graph_step(n, edges, coloring, node_domains=None):
    canvas = nx.Graph()
    for i in range(n):
        canvas.add_node(i)
    for u, v in edges:
        canvas.add_edge(u, v)

    pos = nx.spring_layout(canvas, seed=42)
    plt.figure(figsize=(6,6))
    color_map = ["#DA42A2", "#60CF60", "#4A6EB8", 'yellow', 'purple', 'orange', 'cyan', 'magenta', 'lime', 'red', 'brown', 'pink']
    
    node_colors = []
    for c in coloring:
        if c != -1 and c is not None:
             node_colors.append(color_map[c % len(color_map)])
        else:
             node_colors.append('#888')

    nx.draw(canvas, pos, with_labels=False, node_color=node_colors, node_size=700, edgecolors='black')
    
    for i in range(n):
        node_label = str(i)
        text_color = 'black'
        font_size = 12
        if isinstance(node_domains, (list, tuple)) and i < len(node_domains):
            if coloring[i] == -1:
                domain_set = set(node_domains[i]) if isinstance(node_domains[i], list) else node_domains[i]
                domain_str = ','.join(map(str, sorted(list(domain_set))))
                node_label = f"{i}\n({domain_str})"
        plt.text(pos[i][0], pos[i][1], node_label,
                 fontsize=font_size,
                 ha='center',
                 va='center',
                 fontweight='bold',
                 color=text_color)

    buf = io.BytesIO()
    plt.tight_layout()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_str = base64.b64encode(buf.read()).decode('utf-8')
    plt.close()
    return img_str

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/simulate', methods=['POST'])
def simulate():
    data = request.json
    n = data['nodes']
    edges_raw = data['edges']
    algo = data['algorithm']
    max_colours = data.get('max_colours', n)

    pop_size = data.get('pop_size', 20)
    generations = data.get('generations', 50)
    mutation_rate = data.get('mutation_rate', 0.1)

    graph = [[0]*n for _ in range(n)]
    for u, v in edges_raw:
        graph[u][v] = 1
        graph[v][u] = 1

    solution_data = {}

    try:
        if algo == 'backtracking':
            solution, exec_time, min_colors_found, steps = find_optimal_backtracking(graph, max_colours)
            solution_data = {
                "algorithm": "Backtracking",
                "solution": solution if solution is not None else [-1]*n,
                "time": exec_time,
                "steps": [(s, None) for s in steps],
                "min_colors": min_colors_found
            }

        elif algo == 'backtracking_optimized':
            adj_list = {i: [] for i in range(n)}
            for i in range(n):
                for j in range(n):
                    if graph[i][j] == 1:
                        adj_list[i].append(j)
            
            solution, exec_time, min_colors_found, steps_raw = solve_graph_coloring(adj_list, max_colours, 0)
            
            final_sol = solution if solution is not None else [-1]*n
            
            formatted_steps = [(s, None) for s in steps_raw]

            solution_data = {
                "algorithm": "Optimized Backtracking",
                "solution": final_sol,
                "time": exec_time,
                "steps": formatted_steps,
                "min_colors": min_colors_found
            }

        else:
            ca = CulturalAlgorithm(
                graph,
                population_size=pop_size,
                generations=generations,
                mutation_rate=mutation_rate,
                max_colours=max_colours
            )

            best_solution, best_fitness, exec_time, steps = ca.evolve()
            formatted_steps = [(state, None) for state, fit in steps]

            solution_data = {
                "algorithm": "Cultural Algorithm",
                "solution": best_solution if best_solution is not None else [-1]*n,
                "time": exec_time,
                "steps": formatted_steps,
                "min_colors": calculate_chromatic_number(best_solution)
            }

        final_solution = solution_data['solution']
        solution_data["conflicts"] = calculate_conflicts(graph, final_solution)
        solution_data["chromatic_number"] = calculate_chromatic_number(final_solution)

        return jsonify(solution_data)

    except Exception as e:
        print("Simulation error:", e)
        return jsonify({"error": str(e)}), 500
@app.route('/step_image', methods=['POST'])
def step_image():
    data = request.json
    n = data['nodes']
    edges = data['edges']
    coloring = data['coloring'] 
    node_domains = data.get('node_domains') 
    img_str = plot_graph_step(n, edges, coloring, node_domains)
    return jsonify({'image': img_str})

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [26/Nov/2025 14:33:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:33:50] "GET /stars1.webp HTTP/1.1" 404 -
127.0.0.1 - - [26/Nov/2025 14:34:18] "POST /simulate HTTP/1.1" 200 -
C:\Users\ok\AppData\Local\Temp\ipykernel_27508\2336201661.py:66: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
127.0.0.1 - - [26/Nov/2025 14:34:19] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:20] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:21] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:22] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:22] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:23] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:24] "POST /step_image HTTP/1.1" 200 -
127.0.0.1 - - [26/Nov/2025 14:34:25] "POST /ste